In [6]:
# Interval = min K

import yfinance as yf
from datetime import datetime
import pandas as pd

# ticker = '^gspc'
ticker = 'tsla'

start_date = datetime(2022, 9, 28,21)
end_date = datetime(2022, 9, 29,16)
data = yf.download(ticker, interval = '2m', start = start_date ,end = end_date, auto_adjust = False)
data.index.name = 'Date Time'
data.index = pd.to_datetime(data.index, format = '%m/%d/%Y').strftime('%Y-%m-%d %H:%M:%S')
data.to_csv('data.csv')
print(data)

[*********************100%***********************]  1 of 1 completed
                           Open        High         Low       Close  \
Date Time                                                             
2022-09-28 09:30:00  283.079987  283.140015  281.220001  282.499908   
2022-09-28 09:32:00  280.890015  281.470001  280.299988  280.500000   
2022-09-28 09:34:00  280.420013  280.549988  279.410004  279.769989   
2022-09-28 09:36:00  279.779999  279.959991  278.559998  279.234406   
2022-09-28 09:38:00  279.290009  280.190186  278.850006  278.850006   
...                         ...         ...         ...         ...   
2022-09-28 15:52:00  288.380005  288.880005  288.350006  288.864990   
2022-09-28 15:54:00  288.880005  288.929993  288.019989  288.260010   
2022-09-28 15:56:00  288.269989  288.579987  288.209991  288.339996   
2022-09-28 15:58:00  288.359985  288.369995  287.470001  287.760010   
2022-09-30 16:00:00  265.250000  265.250000  265.250000  265.250000   

       

In [7]:
# Interval = 2 min K

from pyalgotrade import strategy
from pyalgotrade.barfeed import csvfeed
from pyalgotrade.technical import ma
from pyalgotrade.technical import rsi
from pyalgotrade.technical import stoch
from pyalgotrade.bar import Frequency


number_of_shares = 10
kdj = []
kdjd = []
rsi1 = []

def safe_round(value, digits):
    if value is not None:
        value = round(value, digits)
    return value

class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, smaPeriod):
        super(MyStrategy, self).__init__(feed, 100000) # initial equity
        
        # We'll use adjusted close values instead of regular close values.
        # self.setUseAdjustedValues(True)
        self.__sma = ma.SMA(feed[instrument].getCloseDataSeries(), smaPeriod)
        self.__rsi = rsi.RSI(feed[instrument].getCloseDataSeries(), 6)
        self.__kdj = stoch.StochasticOscillator(feed[instrument], period=9, dSMAPeriod=3)
        self.D = self.__kdj.getD()
        self.__position = None
        self.__instrument = instrument

    def onEnterOk(self, position):
        global execInfo
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at $%.2f %s %s %s" % (execInfo.getPrice(),safe_round(self.__kdj[-2], 2),safe_round(self.D[-2], 2),self.__position.getShares()))
        global buy, total_buy
        buy = execInfo.getPrice() * number_of_shares
        total_buy += buy

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("SELL at $%.2f %s %s %s" % (execInfo.getPrice(),safe_round(self.__kdj[-2], 2),safe_round(self.D[-2], 2),self.__position.getShares()))
        # self.__position = None
        sell = execInfo.getPrice() * number_of_shares
        global return_per_share, total_sell
        total_sell += sell
        return_per_share = total_sell - total_buy

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        # Wait for enough bars to be available to calculate a SMA.
        if self.__sma[-1] is None:
            return
        
        kdj.append(self.__kdj[-1])
        kdjd.append(self.D[-1])
        rsi1.append(self.__rsi[-1])
        # for i in enumerate(self.__rsi):
        #     print(i)
        # ALL 30 70 (20)(14)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-2], 2) < 30 and safe_round(self.__rsi[-2], 2) < 30 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterLongStop(self.__instrument, stopPrice=execInfo.getPrice() * 0.995, quantity=number_of_shares, goodTillCanceled=True)
        #     if safe_round(self.__kdj[-2], 2) > 70 and safe_round(self.__rsi[-2], 2) > 70 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterShortStop(self.__instrument, stopPrice=execInfo.getPrice() * 1.005, quantity=number_of_shares, goodTillCanceled=True)
        # else:
        #     if safe_round(self.__kdj[-2], 2) > 70 and safe_round(self.__rsi[-2], 2) > 70 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShortStop(self.__instrument, stopPrice=execInfo.getPrice() * 1.005, quantity=number_of_shares, goodTillCanceled=True)
        #     if safe_round(self.__kdj[-2], 2) < 30 and safe_round(self.__rsi[-2], 2) < 30 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLongStop(self.__instrument, stopPrice=execInfo.getPrice() * 0.995, quantity=number_of_shares, goodTillCanceled=True)


        # ALL 30 70 (13)(15)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-2], 2) < 30 and safe_round(self.__rsi[-2], 2) < 30 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) > 70 and safe_round(self.__rsi[-2], 2) > 70 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-2], 2) > 70 and safe_round(self.__rsi[-2], 2) > 70 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) < 30 and safe_round(self.__rsi[-2], 2) < 30 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # ALL 20 80 (8.6)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # ALL 20 80 
        # if self.__position is None:
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2):
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and \
        #         safe_round(self.__kdj[-1], 2) > safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) > safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20 and \
        #         safe_round(self.__kdj[-1], 2) < safe_round(self.__kdj[-2], 2) and safe_round(self.__rsi[-1], 2) < safe_round(self.__rsi[-2], 2) and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # Just < 30 and > 70 (17)(15)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-1], 2) < 30 and safe_round(self.__rsi[-1], 2) < 30:
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1], 2) > 70 and safe_round(self.__rsi[-1], 2) > 70:
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-1], 2) > 70 and safe_round(self.__rsi[-1], 2) > 70 and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1], 2) < 30 and safe_round(self.__rsi[-1], 2) < 30 and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # Just < 20 and > 80 (10)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-1], 2) < 20 and safe_round(self.__rsi[-1], 2) < 20:
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1], 2) > 80 and safe_round(self.__rsi[-1], 2) > 80:
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-1], 2) > 80 and safe_round(self.__rsi[-1], 2) > 80 and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1], 2) < 20 and safe_round(self.__rsi[-1], 2) < 20 and not self.__position.exitActive() and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>":
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # KDJ cross (15.4)
        # if self.__position is None:
        #     # print(self.__kdj[-1],self.D[-1])
        #     if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2):
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2):
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        if self.__position is None:
            if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2):
                self.__position = self.enterLong(self.__instrument, number_of_shares, True)
            if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2):
                self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        else:
            if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>" and not self.__position.exitActive():
                self.__position.exitMarket()
                self.__position = self.enterShort(self.__instrument, number_of_shares, True)
            if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>" and not self.__position.exitActive():
                self.__position.exitMarket()
                self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # trial 2 (9)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2) and safe_round(self.__kdj[-2], 2) < 50 and safe_round(self.__rsi[-2], 2) < 50:
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2) and safe_round(self.__kdj[-2], 2) > 50 and safe_round(self.__rsi[-2], 2) > 50:
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-2], 2) > 80 and safe_round(self.__rsi[-2], 2) > 80 and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-2], 2) < 20 and safe_round(self.__rsi[-2], 2) < 20 and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)

        # trial 3 (17)
        # if self.__position is None:
        #     if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2) and safe_round(self.__kdj[-2], 2) < 50 and safe_round(self.__rsi[-2], 2) < 50:
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2) and safe_round(self.__kdj[-2], 2) > 50 and safe_round(self.__rsi[-2], 2) > 50:
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        # else:
        #     if safe_round(self.__kdj[-1],2) < safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.LongPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterShort(self.__instrument, number_of_shares, True)
        #     if safe_round(self.__kdj[-1],2) > safe_round(self.D[-1],2) and str(type(self.__position)) == "<class 'pyalgotrade.strategy.position.ShortPosition'>" and not self.__position.exitActive():
        #         self.__position.exitMarket()
        #         self.__position = self.enterLong(self.__instrument, number_of_shares, True)


def run_strategy(smaPeriod):
    global total_sell, total_buy
    total_buy = 0
    total_sell = 0
    # Load the bar feed from the CSV file
    # feed = quandlfeed.Feed()
    feed = csvfeed.GenericBarFeed(Frequency.MINUTE)
    feed.addBarsFromCSV(ticker,'data.csv')

    # Evaluate the strategy with the feed.
    myStrategy = MyStrategy(feed, ticker, smaPeriod)
    myStrategy.run()
    print("Final portfolio value: $%.2f" % myStrategy.getBroker().getEquity())
    # print('Return = ', return_per_share)

run_strategy(15)


2022-09-28 10:00:00 strategy [INFO] BUY at $280.30 68.42 78.91 -10
2022-09-28 10:12:00 strategy [INFO] SELL at $279.75 34.12 14.67 0
2022-09-28 10:12:00 strategy [INFO] BUY at $279.75 34.12 14.67 10
2022-09-28 10:16:00 strategy [INFO] SELL at $279.84 35.0 37.15 0
2022-09-28 10:16:00 strategy [INFO] BUY at $279.84 35.0 37.15 -10
2022-09-28 10:18:00 strategy [INFO] SELL at $280.74 65.88 47.74 0
2022-09-28 10:18:00 strategy [INFO] BUY at $280.74 65.88 47.74 10
2022-09-28 10:26:00 strategy [INFO] SELL at $281.10 81.31 82.96 0
2022-09-28 10:26:00 strategy [INFO] BUY at $281.10 81.31 82.96 -10
2022-09-28 10:28:00 strategy [INFO] SELL at $282.35 98.9 90.48 0
2022-09-28 10:28:00 strategy [INFO] BUY at $282.35 98.9 90.48 10
2022-09-28 10:32:00 strategy [INFO] SELL at $282.64 76.46 91.12 0
2022-09-28 10:32:00 strategy [INFO] BUY at $282.64 76.46 91.12 -10
2022-09-28 10:38:00 strategy [INFO] SELL at $282.96 79.87 77.76 0
2022-09-28 10:38:00 strategy [INFO] BUY at $282.96 79.87 77.76 10
2022-09-28

Final portfolio value: $100317.91


In [10]:
import plotly.graph_objs as go


trace1 = go.Scatter(
    x=data.index,
    y=kdj,
    mode='lines',
    name='KDJ K Line'
)

trace2 = go.Scatter(
    x=data.index,
    y=kdjd,
    mode='lines',
    name='KDJ D Line'
)

trace3 = go.Scatter(
    x=data.index,
    y=rsi1,
    mode='lines',
    name='RSI1 Line'
)

layout = go.Layout(
    title='indicators',
    xaxis={'title': "Time"},
    yaxis={'title': "kdj/rsi"}
)

fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()